# Example: Branched Network Simulation

This notebook demonstrates a simulation of a branched water network controlled by a multi-agent system (MAS). 

The network consists of two reservoirs, each feeding into a channel. These two channels (a main one and a tributary) merge, and the combined flow exits through a final gate. The water levels in the reservoirs are controlled by local PID controllers that adjust the openings of their respective gates.

## System Architecture

### Physical Components
- **Two Reservoirs (`res1`, `res2`)**: The primary water sources.
- **Three Gates (`g1`, `g2`, `g3`)**: `g1` and `g2` are controlled to manage reservoir levels. `g3` is a final, uncontrolled outflow point.
- **Two River Channels (`trib_chan`, `main_chan`)**: Connect the components.

### Multi-Agent System (MAS)
- **Digital Twin Agents**: One for each key physical component to broadcast its state to the message bus.
- **Local Control Agents (LCAs)**: Two LCAs, each equipped with a PID controller. They listen to reservoir level updates and send control actions to their respective gates (`g1` and `g2`).
- **Central Dispatcher**: A central agent that could (in a more complex scenario) override local setpoints based on global system state. Here, it primarily monitors.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.simulation_identification.physical_objects.river_channel import RiverChannel
from swp.core_engine.testing.simulation_harness import SimulationHarness
from swp.local_agents.perception.digital_twin_agent import DigitalTwinAgent
from swp.local_agents.control.pid_controller import PIDController
from swp.local_agents.control.local_control_agent import LocalControlAgent
from swp.central_coordination.dispatch.central_dispatcher import CentralDispatcher

# Simulation configuration
simulation_config = {'duration': 1000, 'dt': 1.0}
harness = SimulationHarness(config=simulation_config)
message_bus = harness.message_bus

# Define physical components
res1 = Reservoir(name="res1", initial_state={'volume': 15e6, 'water_level': 10.0}, parameters={'surface_area': 1.5e6})
g1 = Gate(name="g1", initial_state={'opening': 0.1}, parameters={'width': 10, 'max_rate_of_change': 0.1})
trib_chan = RiverChannel(name="trib_chan", initial_state={'volume': 2e5, 'water_level': 2.0}, parameters={'k': 0.0002})
res2 = Reservoir(name="res2", initial_state={'volume': 30e6, 'water_level': 20.0}, parameters={'surface_area': 1.5e6})
g2 = Gate(name="g2", initial_state={'opening': 0.1}, parameters={'width': 15, 'max_rate_of_change': 0.1})
main_chan = RiverChannel(name="main_chan", initial_state={'volume': 8e5, 'water_level': 8.0}, parameters={'k': 0.0001})
g3 = Gate(name="g3", initial_state={'opening': 0.5}, parameters={'width': 20})

physical_components = [res1, g1, trib_chan, res2, g2, main_chan, g3]
for comp in physical_components:
    harness.add_component(comp)

# Define network topology
harness.add_connection("res1", "g1")
harness.add_connection("g1", "trib_chan")
harness.add_connection("res2", "g2")
harness.add_connection("trib_chan", "main_chan")
harness.add_connection("g2", "main_chan")
harness.add_connection("main_chan", "g3")

# Define the multi-agent control system
twin_agents = [
    DigitalTwinAgent(agent_id="twin_res1", simulated_object=res1, message_bus=message_bus, state_topic="state.res1.level"),
    DigitalTwinAgent(agent_id="twin_g1", simulated_object=g1, message_bus=message_bus, state_topic="state.g1.opening"),
    DigitalTwinAgent(agent_id="twin_res2", simulated_object=res2, message_bus=message_bus, state_topic="state.res2.level"),
    DigitalTwinAgent(agent_id="twin_g2", simulated_object=g2, message_bus=message_bus, state_topic="state.g2.opening"),
]

pid1 = PIDController(Kp=-0.5, Ki=-0.05, Kd=-0.1, setpoint=12.0, min_output=0.0, max_output=1.0)
pid2 = PIDController(Kp=-0.4, Ki=-0.04, Kd=-0.1, setpoint=18.0, min_output=0.0, max_output=1.0)

lca1 = LocalControlAgent(agent_id="lca_g1", controller=pid1, message_bus=message_bus, observation_topic="state.res1.level", observation_key="water_level", action_topic="action.g1.opening", dt=simulation_config['dt'], command_topic="command.res1.setpoint")
lca2 = LocalControlAgent(agent_id="lca_g2", controller=pid2, message_bus=message_bus, observation_topic="state.res2.level", observation_key="water_level", action_topic="action.g2.opening", dt=simulation_config['dt'], command_topic="command.res2.setpoint")

dispatcher = CentralDispatcher(agent_id="central_dispatcher", message_bus=message_bus, state_subscriptions={"res1_level": "state.res1.level", "res2_level": "state.res2.level"}, command_topics={"res1_command": "command.res1.setpoint", "res2_command": "command.res2.setpoint"}, rules={'res1_normal_setpoint': 12.0, 'res2_normal_setpoint': 18.0})

all_agents = twin_agents + [lca1, lca2, dispatcher]
for agent in all_agents:
    harness.add_agent(agent)

# Build and run the MAS simulation
# We will redirect stdout to a file to keep the notebook clean
import sys
original_stdout = sys.stdout
with open('simulation_log.txt', 'w') as f:
    sys.stdout = f
    harness.build()
    harness.run_mas_simulation()
sys.stdout = original_stdout # Restore stdout

print("MAS simulation complete. Results are in harness.history.")

## Results and Visualization

We will now plot the results from the simulation history. The key variables to observe are:
- The water levels in both reservoirs (`res1`, `res2`) to see if they reach their setpoints.
- The openings of the controlled gates (`g1`, `g2`) to see how the controllers are acting.
- The outflow of the final gate (`g3`).

In [ ]:
# Extract data from history
time = [h['time'] for h in harness.history]
res1_levels = [h['res1']['water_level'] for h in harness.history]
res2_levels = [h['res2']['water_level'] for h in harness.history]
g1_openings = [h['g1']['opening'] for h in harness.history]
g2_openings = [h['g2']['opening'] for h in harness.history]
g3_outflows = [h['g3']['outflow'] for h in harness.history]

# Create a DataFrame for easy viewing
df = pd.DataFrame({
    'Time': time,
    'Res1 Level': res1_levels,
    'Res2 Level': res2_levels,
    'G1 Opening': g1_openings,
    'G2 Opening': g2_openings,
    'G3 Outflow': g3_outflows
})

print(df.head())

# Plot the results
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 15), sharex=True)

# Reservoir Levels
ax1.plot(df['Time'], df['Res1 Level'], label='Reservoir 1 Water Level')
ax1.axhline(y=12.0, color='r', linestyle='--', label='Reservoir 1 Setpoint (12.0m)')
ax1.plot(df['Time'], df['Res2 Level'], label='Reservoir 2 Water Level')
ax1.axhline(y=18.0, color='g', linestyle='--', label='Reservoir 2 Setpoint (18.0m)')
ax1.set_ylabel('Water Level (m)')
ax1.set_title('Branched Network Simulation Results')
ax1.grid(True)
ax1.legend()

# Gate Openings
ax2.plot(df['Time'], df['G1 Opening'], label='Gate 1 Opening')
ax2.plot(df['Time'], df['G2 Opening'], label='Gate 2 Opening')
ax2.set_ylabel('Gate Opening (%)')
ax2.grid(True)
ax2.legend()

# Outflow
ax3.plot(df['Time'], df['G3 Outflow'], label='Final Outflow (from g3)', color='purple')
ax3.set_xlabel('Time (s)')
ax3.set_ylabel('Outflow (m^3/s)')
ax3.grid(True)
ax3.legend()

plt.tight_layout()
plt.show()